In [1]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse as sp
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import random
import joblib


In [2]:
HOME = os.getcwd()
print(HOME)
ANIME_DATA = os.path.join(HOME, r'Preprocessing\Preprocessed Data\raw_anime_processed.csv')
print(ANIME_DATA)

d:\Data Science Introduction\Final_Project\models\anime_models
d:\Data Science Introduction\Final_Project\models\anime_models\Preprocessing\Preprocessed Data\raw_anime_processed.csv


In [3]:
# df = pd.read_csv(ANIME_DATA)
df = pd.read_csv(r'D:\Data Science Introduction\Final_Project\Preprocessing\Preprocessed Data\raw_anime_processed.csv')
df.head()

,Title,Score,Vote,Ranked,Popularity,Members,Favorite,Types,Volumes,Chapters,...,Demographic,Serialization,Author,Total Review,Type Review,Realeased date,Completed date,Recommended,Mixed Feelings,Not Recommended
0,Berserk,9.47,363720,1,1,725079,130489,Manga,3,15,...,Seinen,Young Animal,"[""'Miura,Kentarou'"", ""'Studio Gaga'""]",289,"[252, 17, 20]",1989-08-25,Updating,252,17,20
1,JoJo no Kimyou na Bouken Part 7: Steel Ball Ru...,9.31,172219,2,23,280428,46269,Manga,24,96,...,Seinen,Ultra Jump,"[""'Araki,Hirohiko'""]",131,"[123, 7, 1]",2004-01-19,2011-04-19,123,7,1
2,Vagabond,9.26,154583,3,13,406082,44258,Manga,37,327,...,Seinen,Morning,"[""'Inoue,Takehiko'"", ""'Yoshikawa,Eiji'""]",104,"[93, 9, 2]",1998-09-03,2015-05-21,93,9,2
3,One Piece,9.22,392811,4,4,642620,119974,Manga,3,15,...,Shounen,Shounen Jump (Weekly),"[""'Oda,Eiichiro'""]",231,"[190, 21, 20]",1997-07-22,Updating,190,21,20
4,Monster,9.16,104327,5,29,258581,22008,Manga,18,162,...,Seinen,Big Comic Original,"[""'Urasawa,Naoki'""]",86,"[69, 11, 6]",1994-12-05,2001-12-20,69,11,6


In [4]:
df.keys()

Index(['Title', 'Score', 'Vote', 'Ranked', 'Popularity', 'Members', 'Favorite',
       'Types', 'Volumes', 'Chapters', 'Status', 'Published', 'Genres',
       'Themes', 'Demographic', 'Serialization', 'Author', 'Total Review',
       'Type Review', 'Realeased date', 'Completed date', 'Recommended',
       'Mixed Feelings', 'Not Recommended'],
      dtype='object')

In [5]:
df['Genres'] = df['Genres'].fillna(str(['''''']))
df['Themes'] = df['Themes'].fillna(str(['''''']))

In [6]:
# Chuyển các thuộc tính dạng list thành chuỗi văn bản
df['Genres_str'] = df['Genres'].apply(lambda x: " ".join([genre.strip("'") for genre in x.split(',')]) if x and isinstance(x, str) else "")
df['Author_str'] = df['Author'].apply(lambda x: " ".join([author.strip("'") for author in x.split(',')]) if x and isinstance(x, str) else "")

# Áp dụng TF-IDF cho các thuộc tính dạng chuỗi
tfidf = TfidfVectorizer(stop_words='english')

tfidf_Types_matrix = tfidf.fit_transform(df['Types'])
tfidf_Genres_matrix = tfidf.fit_transform(df['Genres_str'])
tfidf_Author_matrix = tfidf.fit_transform(df['Author_str'])

In [7]:

# Chuẩn hóa các thuộc tính dạng số
scaler = MinMaxScaler()
df[['Score', 'Vote', 'Ranked', 'Popularity', 'Recommended']] = scaler.fit_transform(df[['Score', 'Vote', 'Ranked', 'Popularity', 'Recommended']])


In [8]:
X_tfidf = sp.hstack([      # Giữ dạng sparse
    tfidf_Types_matrix,
    tfidf_Genres_matrix,
    tfidf_Author_matrix,
    sp.csr_matrix(df[['Score']].values),  # Chuyển các cột số sang sparse
    sp.csr_matrix(df[['Vote']].values),
    sp.csr_matrix(df[['Ranked']].values),
    sp.csr_matrix(df[['Popularity']].values),
    sp.csr_matrix(df[['Recommended']].values)
])


In [ ]:
# # Tính ma trận độ tương đồng cosine
# cosine_sim = cosine_similarity(X_tfidf, X_tfidf)

# # Save cosine similarity matrix to a file
# joblib.dump(cosine_sim, "cosine_sim.pkl")

# Load it back when needed
cosine_sim = joblib.load("cosine_sim.pkl")

['cosine_sim.pkl']

In [10]:
# Khởi tạo Q-matrix
num_anime = len(df)
Q = np.zeros((num_anime, num_anime))  # Q-matrix: trạng thái và hành động là các chỉ số anime

In [11]:
def q_learning(Q, cosine_sim, num_episodes=1000, alpha=0.1, gamma=0.9, epsilon=0.1):
    for episode in range(num_episodes):
        # Chọn trạng thái ban đầu ngẫu nhiên
        state = random.randint(0, Q.shape[0] - 1)
        
        for step in range(10):  # Số bước tối đa cho mỗi tập
            # Epsilon-greedy: chọn hành động
            if random.uniform(0, 1) < epsilon:
                action = random.randint(0, Q.shape[1] - 1)  # Hành động ngẫu nhiên
            else:
                action = np.argmax(Q[state])  # Hành động tốt nhất theo Q-matrix
            
            # Phần thưởng: Giả lập phản hồi người dùng
            reward = 1 if cosine_sim[state, action] > 0.8 else 0  # Ví dụ: thích nếu tương đồng cao
            
            # Cập nhật Q-value
            next_state = action  # Trong bài toán này, trạng thái kế tiếp là hành động
            Q[state, action] += alpha * (reward + gamma * np.max(Q[next_state]) - Q[state, action])
            
            # Kết thúc nếu đạt phần thưởng cao
            if reward == 1:
                break
                
    return Q


In [ ]:
# Q = q_learning(Q, cosine_sim, num_episodes=500, alpha=0.1, gamma=0.9, epsilon=0.1)

Q = joblib.load("q_matrix.pkl")

In [13]:
def recommend_and_get_feedback(title, Q, df, top_n=5):
    # Lấy chỉ số của anime
    idx = df[df['Title'] == title].index[0]
    
    # Lấy các hành động tốt nhất từ Q-matrix
    recommendations = np.argsort(Q[idx])[-top_n:][::-1]
    recommended_titles = df.iloc[recommendations]['Title'].values
    
    # Hiển thị gợi ý
    print(f"Gợi ý dựa trên '{title}':")
    for i, anime in enumerate(recommended_titles):
        print(f"{i + 1}. {anime}")
    
    # Nhận phản hồi từ người dùng
    feedback = []
    for i in range(top_n):
        response = int(input(f"Bạn có thích '{recommended_titles[i]}'? (1: Thích, 0: Không thích): "))
        feedback.append(response)
    
    return idx, recommendations, feedback


In [14]:
# def update_q_matrix(Q, state, recommendations, feedback, alpha=0.1, gamma=0.9):
#     print(f"Q-value trước khi cập nhật: {Q[state]}")

#     for action, reward in zip(recommendations, feedback):
#         # Cập nhật Q-value theo công thức Q-Learning
#         Q[state, action] += alpha * (reward + gamma * np.max(Q[action]) - Q[state, action])
    
#     # Kiểm tra giá trị Q sau khi cập nhật
#     print(f"Q-value sau khi cập nhật: {Q[state]}")  # In ra các giá trị Q của state
#     return Q


def update_q_matrix(Q, state, recommendations, feedback, alpha=0.5, gamma=0.9, penalty_factor=20):
    # Kiểm tra giá trị Q ban đầu
    print(f"Q-value trước khi cập nhật: {Q[state]}")  # In ra các giá trị Q của state

    for action, reward in zip(recommendations, feedback):
        # Nếu feedback là 0 (không thích), áp dụng một hình phạt mạnh mẽ hơn
        if reward == 0:
            # Tạo hình phạt cho hành động không thích, có thể giảm giá trị Q rất mạnh
            Q[state, action] -= 0.05  # Áp dụng hình phạt mạnh
            Q[state, action] = min(Q[state, action], 0)
            # Q[state, action] -= penalty_factor * alpha * Q[state, action]  # Áp dụng hình phạt mạnh

        # Cập nhật Q-value theo công thức Q-Learning
        else:
            Q[state, action] += alpha * (reward + gamma * np.max(Q[action]) - Q[state, action])

    # Kiểm tra giá trị Q sau khi cập nhật
    print(f"Q-value sau khi cập nhật: {Q[state]}")  # In ra các giá trị Q của state

    return Q



In [15]:
anime_title = "Naruto"  # Anime đầu vào

# Gợi ý và nhận phản hồi
state, recommended_indices, user_feedback = recommend_and_get_feedback(anime_title, Q, df)

# Cập nhật Q-matrix dựa trên phản hồi
Q = update_q_matrix(Q, state, recommended_indices, user_feedback)
# Lưu Q-matrix vào file

# with open("q_matrix.pkl", "w") as file:
#    pass  # This clears the content of the file
joblib.dump(Q, "q_matrix.pkl")

print("\nCập nhật Q-matrix thành công!")


Gợi ý dựa trên 'Naruto':
1. 20th Century Boys
2. Ashita no Joe (Ashita no Joe: Fighting for Tomorrow)
3. Monogatari Series: First Season
4. Mo Dao Zu Shi (Grandmaster of Demonic Cultivation: Mo Dao Zu Shi)
5. Monogatari Series: Second Season
Q-value trước khi cập nhật: [0. 0. 0. ... 0. 0. 0.]
Q-value sau khi cập nhật: [0. 0. 0. ... 0. 0. 0.]

Cập nhật Q-matrix thành công!
